# Experiments

Tiny Towns Scorer\
CS 4664: Data-Centric Computing Capstone

### Authors
Alex Owens, Daniel Schoenbach, Payton Klemens

### Acknowledgements

Portions of this project were adapted from tutorials and examples available
on the [OpenCV](https://opencv.org/).

In particular, significant portions of code from the tutorials [Feature Detection and Description](https://docs.opencv.org/4.6.0/db/d27/tutorial_py_table_of_contents_feature2d.html).

# **Note**
**Unlike the other notebooks, this notebook is not designed to be run in whole. It contains abandoned ideas, half-written snippets, and exploratory code from the lifetime of the project. Much or most of the code has not been tested in this context and may not run as-is.**

# Dependencies

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv

# Data

The collected data has been made available on [Zenodo](https://zenodo.org/record/7429657#.Y5d_np7MKUk).

The dataset is over 1 GB. To avoid re-downloading it each time, the notebook saves it to Google Drive.

In [ ]:
# If you do not want to connect the notebook to your Google Drive,
# simply uncomment this line and comment the three below
# PROJECT_FOLDER = '/content'

# Comment to prevent connecting to Google Drive
from google.colab import drive
drive.mount('/content/drive')
PROJECT_FOLDER = '/content/drive/My Drive/tiny-towns-scorer'

from os import path
IMAGES_FOLDER = path.join(PROJECT_FOLDER, 'images')
ANNOTATIONS_FOLDER = path.join(PROJECT_FOLDER, 'annotations')
MODEL_FOLDER = path.join(PROJECT_FOLDER, 'model', 'model')
CHECKPOINT_PATH = "checkpoint/" # Note: local to runtime environment

In [ ]:
!mkdir -p "{PROJECT_FOLDER}"
!test ! -d "{IMAGES_FOLDER}" && wget -O "images.tar.gz" "https://zenodo.org/record/7429657/files/images.tar.gz?download=1" && tar -xzvf images.tar.gz -C "{PROJECT_FOLDER}"
!test ! -d "{MODEL_FOLDER}" && wget -O "model.tar.gz" "https://zenodo.org/record/7429657/files/model.tar.gz?download=1" && tar -xzvf model.tar.gz -C "{PROJECT_FOLDER}"

## Image loading functions

These are some convience functions for loading images, used by some of the experimental code below. Note that they typically rescale images to at least 1000px on each side (preserving aspect ratio), which the final model does not do.

In [ ]:
from os import path
from glob import glob

IMAGES_FRONTAL = 'frontal'
IMAGES_SIDE = 'side_angle'
IMAGES_TOP_DOWN = 'top_down'

IMAGES_BRICK = 'brick'
IMAGES_CHAPEL = 'chapel'
IMAGES_COTTAGE = 'cottage'
IMAGES_FACTORY = 'factory'
IMAGES_FARM = 'farm'
IMAGES_GLASS = 'glass'
IMAGES_STONE = 'stone'
IMAGES_TAVERN = 'tavern'
IMAGES_THEATER = 'theater'
IMAGES_WELL = 'well'
IMAGES_WHEAT = 'wheat'
IMAGES_WOOD = 'wood'

def list_board_imgs():
  def board(x):
    return path.join(IMAGES_FOLDER, 'boards', x)
  return {
    'full': board('board_scan_full.png'),
    'transparent': board('board_scan_transparent.png'),
    'grid_outline': board('board_scan_grid_outline.png'),
    'grid_blocks': board('board_scan_grid_blocks.png'),
    'frontal': board('board_frontal.png'),
    'top_down': board('board_top_down.png'),
    'side_1': board('board_side_1.png'),
    'side_2': board('board_side_2.png')
  }

def list_game_imgs():
  return {
    angle:
    [ img
      for ext in ['*.JPG', '*.jpeg']
      for img in glob(path.join(IMAGES_FOLDER, angle, ext)) ]
    for angle in [IMAGES_FRONTAL, IMAGES_SIDE, IMAGES_TOP_DOWN]
  }

def list_piece_imgs():
  return {
    piece: glob(path.join(IMAGES_FOLDER, 'pieces', piece, '*.JPG'))
    for piece in [
      IMAGES_BRICK,
      IMAGES_CHAPEL,
      IMAGES_COTTAGE,
      IMAGES_FACTORY,
      IMAGES_FARM,
      IMAGES_GLASS,
      IMAGES_STONE,
      IMAGES_TAVERN,
      IMAGES_THEATER,
      IMAGES_WELL,
      IMAGES_WHEAT,
      IMAGES_WOOD
      ]
  }

def rescale(img, max_dim):
  if img.shape[1] > img.shape[0]:
    width,height = max_dim, int(max_dim * img.shape[0]/img.shape[1])
  else:
    width,height = int(max_dim * img.shape[1]/img.shape[0]), max_dim
  return cv.resize(img, (width, height))

def load_img(name):
  img_path = path.join(PROJECT_FOLDER, 'images', name)
  return rescale(cv.cvtColor(cv.imread(img_path,cv.IMREAD_UNCHANGED),cv.COLOR_BGR2RGB), 1000)

def load_img_gry(name):
  img_path = path.join(PROJECT_FOLDER, 'images', name)
  return rescale(cv.imread(img_path,cv.IMREAD_GRAYSCALE), 1000)


# Manual Corner Detection

Import the packages used for manual corner detection.

In [ ]:
from skimage.io import imread, imshow
from skimage import transform

Load an image and find the corner points manually and perform a projective transformation using skimage and display it.

In [ ]:
side_images = list_game_imgs()
board_ref_file = path.join(IMAGES_FOLDER, 'side_angle/IMG_0237.jpeg')
tt = imread(board_ref_file)
fig, ax = plt.subplots()
ax.imshow(tt)
_ = ax.set_title('original picture')

In [ ]:
# source coords
src = np.array([2166, 705, 
                945, 1251,
                1937, 2325,
                3305, 1334,]).reshape((4, 2))

# dest coords
dst = np.array([1000, 1000,
                1000, 2500,
                2500, 2500,
                2500, 1000,]).reshape((4, 2))

# use skimage's transform module where 'projective' is desired param
tform = transform.estimate_transform('projective', src, dst)
tf_img = transform.warp(tt, tform.inverse)

# plotting the image
fig, ax = plt.subplots()
ax.imshow(tf_img)
_ = ax.set_title('projective transformation')

Chose not to move forward with manual since it seems unoptimal

# Harris Corner Detection


Load an image and attempt to find corners through OpenCV's Harris Corner detection module.

In [ ]:
from google.colab.patches import cv2_imshow

tt = cv.imread(path.join(IMAGES_FOLDER, 'top_down/IMG_4519.JPG'))
gray = cv.cvtColor(tt, cv.COLOR_BGR2GRAY)
gray = np.float32(gray)

# use opencv's harris corner module
harris_corners = cv.cornerHarris(gray, 3, 3, 0.05)
kernel = np.ones((7, 7), np.uint8)
harris_corners = cv.dilate(harris_corners, kernel, iterations=2)

tt[harris_corners > 0.025 * harris_corners.max()] = [255, 127, 127]

# display the resulting corner points detected
cv2_imshow(tt)

The results end up being poor, and don't seem to be consistent.

# Board Matching (ORB)

In [ ]:
# https://docs.opencv.org/4.6.0/d1/d89/tutorial_py_orb.html

board_ref_img = load_img(list_board_imgs()['transparent'])
board_angles = ['transparent', 'top_down', 'frontal', 'side_1', 'side_2', 'grid_outline', 'grid_blocks']
board_imgs = [rescale(load_img(list_board_imgs()[angle]),size) for angle in board_angles for size in [1000]]

orb = cv.ORB_create()
orb.setMaxFeatures(1000)
# orb.setNLevels(16)
# orb.setEdgeThreshold(47)
# orb.setPatchSize(47)
# orb.setFirstLevel(3)
board_ref_kp, board_ref_des = orb.detectAndCompute(board_ref_img,None)
board_kps, board_deses = zip(*(orb.detectAndCompute(img,None) for img in board_imgs))
board_Ms = []
for i in range(len(board_imgs)):
  bf = cv.BFMatcher(normType=cv.NORM_HAMMING)
  matches = bf.knnMatch(board_ref_des,board_deses[i],k=2)
  good = [m for m,n in matches if m.distance < 0.7*n.distance]
  src_pts = np.float32([board_ref_kp[m.queryIdx].pt for m in good]).reshape(-1,1,2)
  dst_pts = np.float32([board_kps[i][m.trainIdx].pt for m in good]).reshape(-1,1,2)
  M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 5.0)
  board_Ms.append(np.linalg.inv(M))
fig,ax = plt.subplots(2,len(board_imgs),figsize=(30,8))
if len(board_imgs) > 1:
  for x in range(len(board_imgs)):
    ax[x // len(board_imgs)][x % len(board_imgs)].imshow(cv.drawKeypoints(board_imgs[x], board_kps[x], None, color=(0,255,0), flags=cv.DRAW_MATCHES_FLAGS_DEFAULT | cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS))
  for x in range(len(board_imgs),2*len(board_imgs)):
    ax[x // len(board_imgs)][x % len(board_imgs)].imshow(cv.warpPerspective(board_imgs[x % 7], board_Ms[x % 7], board_ref_img.shape[:2]))
else:
    ax[0].imshow(cv.drawKeypoints(board_imgs[0], board_kps[0], None, color=(0,255,0), flags=cv.DRAW_MATCHES_FLAGS_DEFAULT | cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS))
    ax[1].imshow(cv.warpPerspective(board_imgs[0], board_Ms[0], board_ref_img.shape[:2]))
plt.show()

In [ ]:
# https://docs.opencv.org/4.6.0/d1/de0/tutorial_py_feature_homography.html

def find_board(board_deses, scene_img, verbose=False):
  MIN_MATCH_COUNT = 10

  scene_kp, scene_des = orb.detectAndCompute(scene_img,None)
  if verbose:
    fig, ax = plt.subplots(1,2,figsize=(10,10))
    ax[0].imshow(scene_img)
    ax[1].imshow(cv.drawKeypoints(scene_img, scene_kp, None, color=(0,255,0), flags=cv.DRAW_MATCHES_FLAGS_DEFAULT))
    plt.show()

  bf = cv.BFMatcher(normType=cv.NORM_HAMMING)

  match = None
  for i,board_des in enumerate(board_deses):
    matches = bf.knnMatch(board_des,scene_des,k=2)
    good = [m for m,n in matches if m.distance < 0.7*n.distance]
    if match is None or len(good) > len(match[1]):
      match = (i,good)

  if len(match[1]) >= MIN_MATCH_COUNT:
    if verbose:
      print(f'Found {len(match[1])} matches')
    return match[0], scene_kp, match[1]
  else:
    if verbose:
      print(f'Not enough matches were found - {len(match[1])}/{MIN_MATCH_COUNT}')
    return None

In [ ]:
game_imgs = list_game_imgs()
{
    angle: sum(1 for img in game_imgs[angle]
                if find_board(board_deses, load_img(img), False))
            / len(game_imgs[angle])
    for angle in game_imgs
}

In [ ]:
import random

# img_dir, img_file = path.split(random.choice([img for angle in [IMAGES_FRONTAL, IMAGES_SIDE, IMAGES_TOP_DOWN] for img in game_imgs[angle]]))
img_dir, img_file = 'side_angle', 'IMG_4517.JPG'
print(path.join(path.basename(img_dir), img_file))
scene_img = load_img(path.join(img_dir, img_file))
# scene_img = load_img('top_down/IMG_0267.jpeg')
match = find_board(board_deses, scene_img, True)
if match is None:
  raise ValueError('No match')
board_idx, scene_kp, good = match

src_pts = np.float32([board_kps[board_idx][m.queryIdx].pt for m in good]).reshape(-1,1,2)
dst_pts = np.float32([scene_kp[m.trainIdx].pt for m in good]).reshape(-1,1,2)

M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 5.0)

if M is not None:
  h,w = board_imgs[board_idx].shape[:2]
  pts = np.float32([ [0,0], [0,h-1], [w-1,h-1], [w-1,0] ]).reshape(-1,1,2)
  dst = cv.perspectiveTransform(pts, M)
  outline_img = cv.polylines(scene_img.copy(),[np.int32(dst)],True,(255,255,255),3,cv.LINE_AA)
  match_img = cv.warpPerspective(scene_img, np.matmul(board_Ms[board_idx], np.linalg.inv(M)), board_ref_img.shape[:2])
  fig, ax = plt.subplots(1,2,figsize=(10,5))
  ax[0].imshow(outline_img)
  ax[1].imshow(match_img)
  plt.show()

draw_params = {# 'matchColor': (0,255,0),
              'singlePointColor': None,
              # 'matchesMask': mask.ravel().tolist(),
              'flags': cv.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS | cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS}
fig,ax = plt.subplots(figsize=(15,15))
plt.imshow(cv.drawMatches(board_imgs[board_idx],board_kps[board_idx],outline_img if M is not None else scene_img,scene_kp,good,None,**draw_params))
plt.show()

# Split Board

In [ ]:
def split_board(img, verbose=False):
  img_h,img_w=img.shape[:2]
  splits = [[None]*4]*4
  if verbose:
    fig,ax = plt.subplots(4,4,figsize=(10,10))
  for x in range(4):
    for y in range(4):
      x0 = int(x*img_h/4.45)
      x1 = int((x+1.33)*(img_h)/4.25)
      y0 = int(y*img_h/4.45)
      y1 = int((y+1.33)*(img_h/4.25))
      splits[x][y] = img[max(y0,0):min(y1,img_h),max(x0,0):min(x1,img_w)]
      if verbose:
        ax[y][x].imshow(splits[x][y])
  return splits

# Individual Pieces Pipeline

This function could be used with the pipelines developed in `training.ipynb` to create a pipeline of individual piece images. The idea was to train a CNN to classify individual pieces, then feed it the extracted portions of a detected grid. This approach was abandoned in favor of an object detection network that classifies pieces and positions at the same time.

In [ ]:
# Converts whole images into a list of all the individual objects
# As above, intended for use with dataset.apply()
# Haven't tested recently, but could be used to train traditional CNN
def to_pieces(bounding_box_format='xywh'):

  def get_pieces(record):
    image = record['images']
    def get_piece(box):
      print(box)
      xmin = tf.cast(tf.math.rint(box[0]), tf.int64)
      ymin = tf.cast(tf.math.rint(box[1]), tf.int64)
      xmax = tf.cast(tf.math.rint(box[0]+box[2]), tf.int64)
      ymax = tf.cast(tf.math.rint(box[1]+box[3]), tf.int64)
      return {
          'image': image[ymin:ymax,xmin:xmax],
          'label': tf.cast(box[4], tf.int64)
      }
    boxes_in_xywh=keras_cv.bounding_box.convert_format(record['bounding_boxes'], bounding_box_format, 'xywh', image)
    return tf.data.Dataset.from_tensor_slices(boxes_in_xywh).map(get_piece)

  def apply(dataset):
    return dataset.flat_map(get_pieces)

  return apply

# Line Detection 1

This attempt at line detection came after the final model was developed and relies on some of the functions in `scorer.ipynb`. It was intended to replace the ORB matching portion of the final model. This code will not run unless a model is trained beforehand.

Helper Methods for Line Detection

In [ ]:
from math import sqrt
from itertools import filterfalse, tee
import keras_cv
from keras_cv import bounding_box
from operator import itemgetter

class_ids = [
  'brick',
  'chapel',
  'cottage',
  'farm',
  'tavern',
  'theater',
  'wheat',
  'wood',
  'board',
  'factory',
  'stone',
  'well',
  'glass',
]
class_mapping = dict(zip(range(len(class_ids)), class_ids))

from math import sqrt
from itertools import filterfalse, tee

# Combination of min and max.
# Give x, returns x such that mn <= x <= mx.
def clamp(x, mn, mx):
  return mn if x < mn else (mx if x > mx else x)

# https://docs.python.org/dev/library/itertools.html#itertools-recipes
# Splits an iterable according to a predicate.
def partition(pred, iterable):
    "Use a predicate to partition entries into false entries and true entries"
    # partition(is_odd, range(10)) --> 0 2 4 6 8   and  1 3 5 7 9
    t1, t2 = tee(iterable)
    return filterfalse(pred, t1), filter(pred, t2)

# Increases the area of a prediction box by a factor.
# The expanded box is clipped to stay inside (0,0) and maxdim.
def expand(box, factor, maxdim):
  xmin, ymin, width, height = box[:4]
  sqrtf = sqrt(factor)
  xmin = max(0, xmin - width*(sqrtf-1)/2)
  ymin = max(0, ymin - height*(sqrtf-1)/2)
  width = min(maxdim[1]-xmin, width * sqrtf)
  height = min(maxdim[0]-ymin, height * sqrtf)
  return [xmin, ymin, width, height, *box[4:]]

# Converts a prediction box to a prediction point.
def center(box):
  return [box[0]+box[2]/2, box[1]+box[3]/2, *box[4:]]

# Given a prediction box, returns a predicate
# that tests if a point is inside that box.
def inside(board_pred):
  xmin, ymin, width, height = board_pred[:4]
  def apply(pt):
    x,y = pt[:2]
    return xmin <= x <= xmin+width and ymin <= y <= ymin + height
  return apply

# Given a prediction box, returns a function
# that translates point coordinates to be relative to the box.
def translate(board_pred):
  xmin, ymin = board_pred[:2]
  def apply(pt):
    x,y = pt[:2]
    return [x-xmin, y-ymin, *pt[2:]]
  return apply

# Given a prediction box, returns a function
# that translates point coordinates to be relative to the box.
def translate(board_pred):
  xmin, ymin = board_pred[:2]
  def apply(pt):
    x,y = pt[:2]
    return [x-xmin, y-ymin, *pt[2:]]
  return apply

# Given an image and prediction box,
# returns the portion of the image contained by the box.
def extract(image, box):
  xmin = round(box[0])
  ymin = round(box[1])
  xmax = round(box[0]+box[2])
  ymax = round(box[1]+box[3])
  return image[ymin:ymax,xmin:xmax]

# Given an area, returns a function
# that classifies points in that area into a 4x4 grid.
def gridify(width, height):
  def apply(pt):
    x,y = pt[:2]
    return [clamp(int(x // (width/4)),0,3), clamp(int(y // (height/4)),0,3), *pt[2:]]
  return apply

IMAGE_SIZE = 1000
model = None # A model needs to be created for this code to run
def get_predictions(image):
  img = cv.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
  preds = model.predict(np.array([img]))[0]
  rel_pred = bounding_box.convert_format(preds, 'xywh', 'rel_xyxy', img)
  preds = bounding_box.convert_format(rel_pred, 'rel_xyxy', 'xywh', image)
  return preds.numpy()

In [ ]:


image = cv.cvtColor(cv.imread(path.join(IMAGES_FOLDER, 'side_angle/IMG_6212.jpg')), cv.COLOR_BGR2RGB)
# image = cv.cvtColor(cv.imread(path.join(IMAGES_FOLDER, 'side_angle/IMG_4557.JPG')), cv.COLOR_BGR2RGB)
# image = cv.cvtColor(cv.imread(path.join(IMAGES_FOLDER, 'side_angle/IMG_0280.jpeg')), cv.COLOR_BGR2RGB)

predictions = get_predictions(image)
_, board_preds = partition(lambda b: class_mapping[b[4]] == 'board', predictions)
board_pred = next(iter(sorted(board_preds, key=itemgetter(5), reverse=True)), None)
if board_pred is None:
  raise ValueError('Board could not be found')
board_pred = expand(board_pred, 1.25, image.shape[:2])
board_img, _ = extract(image, board_pred)

_, thresh = cv.threshold(cv.cvtColor(board_img, cv.COLOR_RGB2GRAY), 0, 255, cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
fig, ax = plt.subplots(1,1,figsize=(8,8))
ax.axis('off')
ax.imshow(thresh, cmap='gray')
plt.show()

lines = [line[0] for line in cv.HoughLinesP(thresh,1,np.pi/180,100,minLineLength=500,maxLineGap=10)]
length = lambda line: sqrt((line[2]-line[0])**2+(line[3]-line[1])**2)
dist = lambda pt1, pt2: length([*pt1, *pt2])
slope = lambda line: (line[3]-line[1])/(line[2]-line[0])
slope_intercept = lambda line: (slope(line), line[1]-slope(line)*line[0])
dot = lambda line1, line2: (line1[2]-line1[0])*(line2[2]-line2[0])+(line1[3]-line1[1])*(line2[3]-line2[1])
def intersect(line1, line2):
  m,b = slope_intercept(line1)
  n,c = slope_intercept(line2)
  x = (b-c)/(n-m)
  y = m*x+b
  return round(x),round(y)
# https://www.math.utah.edu/~treiberg/Perspect/Perspect.htm
def fourth_point(o, ap, fp): # ap = a', fp = f'
  f = [o[0] + dist(o, fp), o[1]]
  x = (pt2[0]+pt3[0])/2
  y = (pt2[1]+pt3[1])/2
  return [round(x), round(y)]

lines = sorted(lines, key=length, reverse=True)
longest = lines[0]
perp = min(lines, key=lambda line: dot(longest, line))
longest_perp = next(line for line in lines if abs(slope(perp) - slope(line)) < 0.5)
pt1 = intersect(longest, longest_perp)
dist_pt1 = lambda pt: dist(pt, pt1)
pt2 = max([longest[:2], longest[2:]], key=dist_pt1)
pt3 = max([longest_perp[:2], longest_perp[2:]], key=dist_pt1)
pt4 = fourth_point(pt1, pt2, pt3)

h,w = board_ref_img.shape[:2]
src_pts = np.float32([[0,0], [0,h-1], [w-1,0], [w-1,h-1]]).reshape(-1,1,2)
dst_pts = np.float32([pt1, pt2, pt3, pt4]).reshape(-1,1,2)
homography, _ = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 5.0)

canvas = board_img.copy()
for line in [longest, longest_perp, [*pt2, *pt4], [*pt3, *pt4]]: # [[*pt1, *pt2], [*pt1, *pt3]]:
  x1,y1,x2,y2 = line
  cv.line(canvas,(x1,y1),(x2,y2),(128,255,255),20)
fig, ax = plt.subplots(1,1,figsize=(8,8))
ax.axis('off')
ax.imshow(canvas)
plt.show()

# fig, ax = plt.subplots(1,1,figsize=(8,8))
# ax.axis('off')
# ax.imshow(cv.warpPerspective(board_img, homography, board_ref_img.shape[:2]))
# plt.show()

# Line Detection 2

This attempt at line detection also came during development of the final model.

In [ ]:
import math
def findAngle(line1, line2):
  (x11, y11, x21, y21) = line1
  vec1 = np.array([(y21 - y11) / (x21 - x11), 1])
  (x12, y12, x22, y22) = line2
  vec2 = np.array([(y22 - y12) / (x22 - x12), 1])
  x = vec1.dot(vec2) / math.sqrt(vec1.dot(vec1) * vec2.dot(vec2))
  angle = math.acos(x)*180 / np.pi 
  return angle

def find_piece_grid(board_img, piece_preds):
  centers = [center(p) for p in piece_preds]
  max_dist = board_img.shape[0] // 5
  min_dist = round(max_dist * math.sqrt(2) / 2)
  remove_idxs = []
  for i in range(len(centers)):
    for j in range(i):
      x1, y1 = centers[i]
      x2, y2 = centers[j]
      distance = math.sqrt(((x2 - x1) ** 2) + ((y2 - y1) ** 2))
      if distance < min_dist:
        c1 = piece_preds[i][5]
        c2 = piece_preds[j][5]
        remove_idxs.append(i if c1 < c2 else j)
  remove_idxs = [*set(remove_idxs)] # get rid of dupes
  print(len(centers))
  clean_centers = []
  for i in range(len(centers)):
    if i in remove_idxs:
      continue
    clean_centers.append(centers[i])
  print(len(clean_centers))
  lines = []
  for i in range(len(clean_centers)):
    for j in range(i):
      lines.append((clean_centers[i][0], clean_centers[i][1], clean_centers[j][0], clean_centers[j][1]))

  print(lines)
  
  # print(max_dist, min_dist)
  grid = [[(None, 0) for _ in range(4)] for _ in range(4)]
  return grid

# Line Detection 3

This code uses Hough Line detection to find perpendicular lines in a given image, and use those lines to create the most likely 4x4 grid.

In [ ]:
import time
img = cv.imread(path.join(IMAGES_FOLDER, 'top_down/IMG_4519.JPG'))
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
imshow(gray)
alpha = 1.5
beta = 50
adjusted_gray = cv.convertScaleAbs(gray, alpha=alpha, beta=beta)
imshow(adjusted_gray)
inv_gray = cv.bitwise_not(adjusted_gray)
imshow(inv_gray)
kernel_size = 5
blur_gray = cv.GaussianBlur(inv_gray,(kernel_size, kernel_size),0)
imshow(blur_gray)
# for i in range(1, 21):
#   for j in range(i - 1):
#     low_threshold = j * 10
#     high_threshold = i * 10
#     edges = cv.Canny(blur_gray, low_threshold, high_threshold)
#     imshow(edges)
#     time.sleep(.2)
low_threshold = 20
high_threshold = 100
edges = cv.Canny(blur_gray, low_threshold, high_threshold)
imshow(edges)



rho = 1  # distance resolution in pixels of the Hough grid
theta = np.pi / 180  # angular resolution in radians of the Hough grid
threshold = 30  # minimum number of votes (intersections in Hough grid cell)
min_line_length = 250  # minimum number of pixels making up a line
max_line_gap = 50  # maximum gap in pixels between connectable line segments
line_image = np.copy(img) * 0  # creating a blank to draw lines on

# Run Hough on edge detected image
# Output "lines" is an array containing endpoints of detected line segments
lines = cv.HoughLinesP(edges, rho, theta, threshold, np.array([]),
                    min_line_length, max_line_gap)

for line in lines:
  for x1,y1,x2,y2 in line:
    # print(line)
    cv.line(line_image,(x1,y1),(x2,y2),(255,0,0),5)

# # Draw the lines on the  image
lines_edges = cv.addWeighted(img, 0.8, line_image, 1, 0)
lines_edges
imshow(line_image)

In [ ]:
import math
# def isPerpendicular(line1, line2, factor):
#   (x11, y11, x21, y21) = line1
#   vec1 = np.array([(y21 - y11) / (x21 - x11), 1])
#   (x12, y12, x22, y22) = line2
#   vec2 = np.array([(y22 - y12) / (x22 - x12), 1])
#   x = vec1.dot(vec2) / math.sqrt(vec1.dot(vec1) * vec2.dot(vec2))
#   angle = math.acos(x) *180 / np.pi
#   if angle < 90 + factor and angle > 90 - factor:
#     return True
#   return False
# def isParallel(line1, line2, factor):
#   (x11, y11, x21, y21) = line1
#   vec1 = np.array([(y21 - y11) / (x21 - x11), 1])
#   (x12, y12, x22, y22) = line2
#   vec2 = np.array([(y22 - y12) / (x22 - x12), 1])
#   x = vec1.dot(vec2) / math.sqrt(vec1.dot(vec1) * vec2.dot(vec2))
#   angle = math.acos(x)*180 / np.pi 
#   print(angle)
#   print(factor)
#   if angle < factor:
#     True
#   return False

def findAngle(line1, line2):
  (x11, y11, x21, y21) = line1
  vec1 = np.array([(y21 - y11) / (x21 - x11), 1])
  (x12, y12, x22, y22) = line2
  vec2 = np.array([(y22 - y12) / (x22 - x12), 1])
  x = vec1.dot(vec2) / math.sqrt(vec1.dot(vec1) * vec2.dot(vec2))
  angle = math.acos(x)*180 / np.pi 
  return angle

def is_arr_in_list(myarr, list_arrays):
  return next((True for elem in list_arrays if elem is myarr), False)

In [ ]:
perpendicular_factor = 10
vertical_factor = 0.1
parallel_factor = 12

perpendicular_lines = []
for i in range(len(lines)):
  perpendicular_lines.append(0)
  for j in range(i + 1, len(lines)):
    angle = findAngle(lines[i][0], lines[j][0])
    if angle > 90 - perpendicular_factor and angle < 90 + perpendicular_factor:
      perpendicular_lines[i] += 1


In [ ]:
print(perpendicular_lines)

In [ ]:
best_line = lines[perpendicular_lines.index(max(perpendicular_lines))][0]
print(best_line)
parallels = []
perpendiculars = []
for line in lines:
  angle = findAngle(best_line, line[0])
  if angle < parallel_factor:
    parallels.append(line)
  elif angle > 90 - perpendicular_factor and angle < 90 + perpendicular_factor:
    perpendiculars.append(line)

In [ ]:
par_slopes = 0

for line in parallels:
  (x1, y1, x2, y2) = line[0]
  par_slopes += (y2 - y1) / (x2 - x1)

par_slopes_avg = par_slopes / len(parallels)
par_T = np.linalg.inv(np.array([[1, par_slopes_avg], [par_slopes_avg, -1]]))

perp_slopes = 0
for line in perpendiculars:
  (x1, y1, x2, y2) = line[0]
  perp_slopes += (y2 - y1) / (x2 - x1)

perp_slopes_avg = perp_slopes / len(perpendiculars)
perp_T = np.linalg.inv(np.array([[1, perp_slopes_avg], [perp_slopes_avg, -1]]))

par_x_values = []
par_y_values = []
for line in parallels:
  (x1, y1, x2, y2) = line[0]
  mid = np.array([(x2 + x1) / 2, (y2 + y1) / 2])
  par_x_values.append(par_T.dot(mid)[0])
  par_y_values.append(par_T.dot(mid)[1])

perp_x_values = []
perp_y_values = []
for line in perpendiculars:
  (x1, y1, x2, y2) = line[0]
  mid = np.array([(x2 + x1) / 2, (y2 + y1) / 2])
  perp_x_values.append(perp_T.dot(mid)[0])
  perp_y_values.append(perp_T.dot(mid)[1])

In [ ]:
print(par_slopes_avg)

In [ ]:
!pip install ckwrap

In [ ]:
import ckwrap
import sys
k = 12
par_km = ckwrap.ckmeans(par_y_values, k)
perp_km = ckwrap.ckmeans(perp_y_values, k)

par_labels = par_km.labels
perp_labels = perp_km.labels

# print(par_km.centers)
par_min_err = float("inf")
par_min_err_clusters = []
par_delta = 0
perp_min_err = float("inf")
perp_min_err_clusters = []
perp_delta = 0
for i1 in range(4, k):
  for i2 in range(3, i1):
    for i3 in range(2, i2):
      for i4 in range(1, i3):
        for i5 in range(i4):
          par_centers = [par_km.centers[i1], par_km.centers[i2], par_km.centers[i3], par_km.centers[i4], par_km.centers[i5]]
          par_centers.sort()
          par_diffs = [par_centers[1] - par_centers[0], par_centers[2] - par_centers[1], par_centers[3] - par_centers[2], par_centers[4] - par_centers[3]]
          par_errs = []
          for i in range(len(par_diffs)):
            for j in range(i + 1, len(par_diffs)):
              par_errs.append(abs(par_diffs[i] - par_diffs[j]))
          par_err = sum(par_errs) / len(par_errs)
          # print(err)
          if par_err < par_min_err:
            par_delta = sum(par_diffs) / len(par_diffs)
            par_min_err = par_err
            par_min_err_clusters = [i1, i2, i3, i4, i5]

          perp_centers = [perp_km.centers[i1], perp_km.centers[i2], perp_km.centers[i3], perp_km.centers[i4], perp_km.centers[i5]]
          perp_centers.sort()
          perp_diffs = [perp_centers[1] - perp_centers[0], perp_centers[2] - perp_centers[1], perp_centers[3] - perp_centers[2], perp_centers[4] - perp_centers[3]]
          perp_errs = []
          for i in range(len(perp_diffs)):
            for j in range(i + 1, len(perp_diffs)):
              perp_errs.append(abs(perp_diffs[i] - perp_diffs[j]))
          perp_err = sum(perp_errs) / len(perp_errs)
          # print(err)
          if perp_err < perp_min_err:
            perp_delta = sum(perp_diffs) / len(perp_diffs)
            perp_min_err = perp_err
            perp_min_err_clusters = [i1, i2, i3, i4, i5]


In [ ]:
print(par_min_err_clusters)
for i in range(5):
  print(par_km.centers[par_min_err_clusters[i]])
print(par_delta)

In [ ]:

from matplotlib import pyplot as pltimport
 
 
# Creating dataset
 
# Creating histogram
fig, ax = plt.subplots(figsize =(10, 7))
# ax.plot(perp_x_values, perp_y_values, 'bo')
ax.plot(par_x_values, par_y_values, 'ro')
# ax.hist(par_x_values, bins = 30)
# ax.hist(perp_y_values, bins = 30)
# ax.hist(par_y_values, bins = 30)

# Show plot
plt.show()

This last code cell prints the most likely parallel and perpendicular lines.

In [ ]:
curr_lab = 9
line_image_2 = np.copy(img) * 0  # creating a blank to draw lines on
# for i in range(len(parallels)):
#   line = parallels[i]
#   for x1,y1,x2,y2 in line:
#     # print(line)
#     label = par_labels[i]
#     if label != curr_lab:
#       continue
#     # if label not in par_min_err_clusters:
#     #   continue
#     pt1 = par_T.dot(np.array([x1,y1]))
#     pt1t = (round(pt1[0])+ 1200, round(pt1[1]) + 1600)
#     pt2 = par_T.dot(np.array([x2,y2]))
#     pt2t = (round(pt2[0])+ 1200, round(pt2[1])+ 1600)
#     # print(pt1, pt2)
#     cv.line(line_image_2,pt1t,pt2t,(255,0,0),5)
#     cv.line(line_image_2, [x1, y1], [x2, y2],(255,0,0),5)
# for i in range(len(perpendiculars)):
#   line = perpendiculars[i]
#   for x1,y1,x2,y2 in line:
#     # print(line)
#     label = perp_labels[i]
#     # if label != curr_lab:
#     #   continue
#     if label not in perp_min_err_clusters:
#       continue
#     pt1 = perp_T.dot(np.array([x1,y1]))
#     pt1t = (round(pt1[0]) + 500, round(pt1[1]) + 1200)
#     pt2 = perp_T.dot(np.array([x2,y2]))
#     pt2t = (round(pt2[0]) + 500, round(pt2[1]) + 1200)
#     # print(pt1, pt2)
#     cv.line(line_image_2,pt1t,pt2t,(0,255,0),5)
#     cv.line(line_image_2, [x1, y1], [x2, y2],(255,0,0),5)
for i in range(len(parallels)):
  line = parallels[i]
  for x1,y1,x2,y2 in line:
    # print(line)
    label = par_labels[i]
    # if label != curr_lab:
    #   continue
    # if label not in par_min_err_clusters:
    #   continue
    cv.line(line_image_2,(x1,y1),(x2,y2),(255,0,0),5)

for i in range(len(perpendiculars)):
  line = perpendiculars[i]
  for x1,y1,x2,y2 in line:
    # print(line)
    label = perp_labels[i]
    # if label != curr_lab:
    #   continue
    # if label in perp_min_err_clusters:
    #   continue
    cv.line(line_image_2,(x1,y1),(x2,y2),(0,255,0),5)

for (x1, y1, x2, y2) in lines[perpendicular_lines.index(max(perpendicular_lines))]:
  cv.line(line_image_2, (x1,y1),(x2,y2),(0,255,0),5)
# # Draw the lines on the  image
# lines_edges_2 = cv.addWeighted(img, 0.8, line_image, 1, 0)
# lines_edges_2
imshow(line_image_2)